In [1]:
!pwd


/content


In [2]:
#unzip and extract data files
from zipfile import ZipFile
file_name = "titanic.zip"

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print("Done")

Done


In [3]:
#load the train data
import pandas as pd
df_train = pd.read_csv("train.csv")
df_train.head(5)

df_test = pd.read_csv("test.csv")
df_test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
#converting sex to integers
df_train['Sex']= df_train['Sex'].map({"female":1 , "male":0})
df_test['Sex']= df_test['Sex'].map({"female":1 , "male":0})

In [5]:
#extract the titles from the names and label
df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# df_train['Title'].value_counts()
# print(df_test['Title'].value_counts())

titles = {"Mr": 1, "Mrs": 2, "Miss": 3, "Master": 4, "Ms":3, "Lady":3, "Countess":3, "Mme": 2, 
          "Mlle":3, "Dr":5, "Rev":5, "Col":5, "Major":5, "Don":5, "Capt":5, "Sir":5, "Jonkheer":5, "Dona":2 }   
df_train['Title']= df_train['Title'].map(titles)
df_test['Title']= df_test['Title'].map(titles)

print(df_train['Title'].value_counts())
print(df_test['Title'].value_counts())


1    517
3    187
2    126
4     40
5     21
Name: Title, dtype: int64
1    240
3     79
2     73
4     21
5      5
Name: Title, dtype: int64


In [6]:
#Assign the median value of each title category to fill missing age values

df_train.groupby('Title')['Age'].agg(['mean','median', 'min','max'])
df_train['Age'].fillna(df_train.groupby('Title')['Age'].transform('median'), inplace=True)
df_test['Age'].fillna(df_train.groupby('Title')['Age'].transform('median'), inplace=True)


In [7]:
#Check the class of missing Embarked passegers
filt = (df_train['Embarked']=='S') | (df_train['Embarked']=='Q') | (df_train['Embarked']=='C')
df_train.loc[~filt]

#Assign S for those missing value
df_train['Embarked'].fillna('S', inplace=True)


In [8]:
#display the fare prices for Pclass
print(df_train.groupby('Pclass')['Fare'].agg(['mean','median', 'min','max']))
print('------------')
print(df_test.groupby('Pclass')['Fare'].agg(['mean','median', 'min','max']))

             mean   median  min       max
Pclass                                   
1       84.154687  60.2875  0.0  512.3292
2       20.662183  14.2500  0.0   73.5000
3       13.675550   8.0500  0.0   69.5500
------------
             mean   median     min       max
Pclass                                      
1       94.280297  60.0000  0.0000  512.3292
2       22.202104  15.7500  9.6875   73.5000
3       12.459678   7.8958  3.1708   69.5500


In [9]:
df_test['Fare'].fillna(df_train.groupby('Pclass')['Fare'].transform('mean'), inplace=True)

In [ ]:
#new column cabin_deck with the deck name exrtracted from cabin
df_train['Cabin_deck'] = df_train['Cabin'].str[:1]
df_test['Cabin_deck'] = df_test['Cabin'].str[:1]

print(df_train['Cabin_deck'].value_counts())
print(df_test['Cabin_deck'].value_counts())


In [11]:
#Assign 'N' for all the missing cabin deck values
df_train['Cabin_deck'].fillna(value='N', inplace=True)
df_test['Cabin_deck'].fillna(value='N', inplace=True)

#map the cabin values to intergers
deck = {"T": 0, "A": 1, "B": 2, "C": 3, "D": 4, "E":5, "F":6, "G":7, "N":8}
      
df_train['Cabin_deck']= df_train['Cabin_deck'].map(deck)
df_test['Cabin_deck']= df_test['Cabin_deck'].map(deck)


In [12]:
#mapping Embarked coumn to integers
Embark = {'S':1, 'C':2, 'Q':3}

df_train['Embarked']= df_train['Embarked'].map(Embark)
df_test['Embarked']= df_test['Embarked'].map(Embark)


In [13]:
#droping unwanted columns [Name, Ticket, Cabin, Ticket_pref]
df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)



In [14]:
#converting age to groups
data = [df_train, df_test]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6


In [15]:
#converting fare values to integers
df_train['Fare'] = df_train['Fare'].astype(int)
df_test['Fare'] = df_test['Fare'].astype(int)

In [16]:
data = [df_train, df_test]
for dataset in data:
    dataset['family'] = dataset['SibSp']+ dataset['Parch']
    dataset['Age_Class'] = dataset['Age']* dataset['Pclass']

In [17]:
#get the quantilevalues for Fare for grouping
fare_arr = df_train['Fare'].values
pd.qcut(fare_arr,6)

[(-0.001, 7.0], (52.0, 512.0], (-0.001, 7.0], (52.0, 512.0], (7.0, 8.0], ..., (8.0, 14.0], (26.0, 52.0], (14.0, 26.0], (26.0, 52.0], (-0.001, 7.0]]
Length: 891
Categories (6, interval[float64]): [(-0.001, 7.0] < (7.0, 8.0] < (8.0, 14.0] < (14.0, 26.0] <
                                    (26.0, 52.0] < (52.0, 512.0]]

In [18]:
#group the Fare based on 6 quantiles
data = [df_train, df_test]
for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7, 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 7) & (dataset['Fare'] <= 8), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 8) & (dataset['Fare'] <= 14), 'Fare'] = 3
    dataset.loc[(dataset['Fare'] > 14) & (dataset['Fare'] <= 26), 'Fare'] = 4
    dataset.loc[(dataset['Fare'] > 26) & (dataset['Fare'] <= 52), 'Fare'] = 5
    dataset.loc[dataset['Fare'] > 52, 'Fare'] = 6 


In [19]:
df_test.head(3)
df_train.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_deck,family,Age_Class
0,1,0,3,0,2,1,0,1,1,1,8,1,6
1,2,1,1,1,5,1,0,6,2,2,3,1,5
2,3,1,3,1,3,0,0,1,1,3,8,0,9


In [20]:
#X and y selection
X= df_train.drop(['Survived','PassengerId'], axis=1)
y = df_train.Survived

In [21]:
import tensorflow as tf
import keras


Using TensorFlow backend.


In [22]:
device= tf.test.gpu_device_name()
print(device)


/device:GPU:0


In [26]:
# df.dtypes
X.info()
X.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Pclass      891 non-null    int64
 1   Sex         891 non-null    int64
 2   Age         891 non-null    int64
 3   SibSp       891 non-null    int64
 4   Parch       891 non-null    int64
 5   Fare        891 non-null    int64
 6   Embarked    891 non-null    int64
 7   Title       891 non-null    int64
 8   Cabin_deck  891 non-null    int64
 9   family      891 non-null    int64
 10  Age_Class   891 non-null    int64
dtypes: int64(11)
memory usage: 76.7 KB


(891, 11)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam, RMSprop


In [34]:
# create the sequential model
tf.random.set_seed(0)


model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(11,)))
model.add(Dropout(0.2))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                240       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 30)                630       
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________


In [37]:
#Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x= X, y= y, epochs =200)






Epoch 1/200
891/891 [==============================] - 0s 346us/step - loss: 0.3761 - accuracy: 0.8384
Epoch 2/200
891/891 [==============================] - 0s 162us/step - loss: 0.3792 - accuracy: 0.8429
Epoch 3/200
891/891 [==============================] - 0s 147us/step - loss: 0.3806 - accuracy: 0.8451
Epoch 4/200
891/891 [==============================] - 0s 146us/step - loss: 0.3798 - accuracy: 0.8361
Epoch 5/200
891/891 [==============================] - 0s 147us/step - loss: 0.3777 - accuracy: 0.8350
Epoch 6/200
891/891 [==============================] - 0s 147us/step - loss: 0.3872 - accuracy: 0.8418
Epoch 7/200
891/891 [==============================] - 0s 165us/step - loss: 0.3909 - accuracy: 0.8316
Epoch 8/200
891/891 [==============================] - 0s 160us/step - loss: 0.3725 - accuracy: 0.8485
Epoch 9/200
891/891 [==============================] - 0s 193us/step - loss: 0.3762 - accuracy: 0.8496
Epoch 10/200
891/891 [==============================] - 0s 160us/step - l

In [38]:
#prepare the test data for prediction
df_test.head(3)
passenger_Id = df_test['PassengerId']
X_test_final = df_test.drop('PassengerId', axis =1)

In [45]:
y_pred = model.predict_classes(X_test_final)


In [58]:
# rounded = [round(x[0]) for x in y_pred]
# print(rounded)
y_pred_final = y_pred.reshape(1,-1)


[0]


In [59]:
#get the answer to a df with passengerID and survived columns
y_pred_series = pd.Series(y_pred_final[0], name='Survived')
answer_df= pd.concat([passenger_Id, y_pred_series], axis =1)


In [62]:
from google.colab import files
answer_df.to_csv('titanic_keras.csv', index=False) 
files.download('titanic_keras.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>